# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."
import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
Using old version of models
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = '843.9765.A73.AK4'

# the auction goes:
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 1, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 2, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 3, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': 'PASS', 'insta_score': 0.832}, {'call': '1C', 'insta_score': 0.168}]
[{'call': 'PASS', 'insta_score': 0.786}, {'call': '1C', 'insta_score': 0.213}]
[{'call': 'PASS', 'insta_score': 0.832}, {'call': '1C', 'insta_score': 0.168}]
[{'call': 'PASS', 'insta_score': 0.786}, {'call': '1C', 'insta_score': 0.213}]


in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
bid.samples

[]

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
8xx.9xxx.Axx.AKx AQT9xx.JTx.9.Jxx .AKQx.Jxxxx.T9xx KJxx.8x.KQT8.Q8x
8xx.9xxx.Axx.AKx QJT9xx.xx.KJTx.x .AKJx.Q98xx.Q8xx AKxx.QT8.x.JT9xx
8xx.9xxx.Axx.AKx KQJTxx.8.QJ9.Jxx .AKxx.K8xxx.98xx A9xx.QJTx.Tx.QTx
8xx.9xxx.Axx.AKx KJT9x.Qx.9x.J8xx x.AK8x.KJ8x.Q9xx AQxx.JTx.QTxx.Tx
8xx.9xxx.Axx.AKx KQJTxx.Jxx.Jx.T8 .AKQx.KQ9x.Q9xxx A9xx.T8.T8xx.Jxx
8xx.9xxx.Axx.AKx AKQ9.Jx.Q9xx.Jxx x.AKTx.KTxx.QT8x JTxxx.Q8x.J8.9xx
8xx.9xxx.Axx.AKx KJxxx.JTx.Txx.8x .AKQ8.KQJ.QJ9xxx AQT9x.xx.98xx.Tx
8xx.9xxx.Axx.AKx AQJT9.Tx.JTx.JTx x.AJ8x.KQ8x.Q8xx Kxxx.KQx.9xx.9xx
8xx.9xxx.Axx.AKx AJT9xx.8xx.QTx.x .AKJT.K9xx.9xxxx KQxx.Qx.J8x.QJT8
8xx.9xxx.Axx.AKx AT9xxx.QJ.T8x.xx x.AKT8.KQxx.J98x KQJ.xxx.J9x.QTxx
8xx.9xxx.Axx.AKx AKJx.8xxx.Jx.J9x x.AKJT.K8xx.QT8x QT9xx.Q.QT9x.xxx
8xx.9xxx.Axx.AKx T9xxx.Jxx.Txx.Jx K.AKQT.KQ8x.Q8xx AQJx.8x.J9x.T9xx
8xx.9xxx.Axx.AKx AT9xx.xx.98x.Qxx K.AKQ8.QJTx.T9xx QJxx.JTx.Kxx.J8x
8xx.9xxx.Axx.AKx AQTxx.Jxx.QJ.T8x 9.AKQT.KTxx.QJ9x KJxx.8x.98xx.xxx
8xx.9xxx.Axx.AKx QJ9xx.Jx.K98